In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
from  datetime import datetime, timedelta
import gc
import numpy as np, pandas as pd
import lightgbm as lgb

In [ ]:
CAL_DTYPES={"event_name_1": "category", "event_name_2": "category", "event_type_1": "category", 
         "event_type_2": "category", "weekday": "category", 'wm_yr_wk': 'int16', "wday": "int16",
        "month": "int16", "year": "int16", "snap_CA": "float32", 'snap_TX': 'float32', 'snap_WI': 'float32' }
PRICE_DTYPES = {"store_id": "category", "item_id": "category", "wm_yr_wk": "int16","sell_price":"float32" }

In [ ]:
pd.options.display.max_columns = 50

In [ ]:
# import datetime
# h = 28 
# max_lags = 57
# tr_last = 1941 #1913
# fday = datetime.datetime(2016,5,24) #(2016,4, 25) 
# fday

In [ ]:
# #create dt
# sales = pd.read_csv("../input/m5-forecasting-accuracy/sales_train_evaluation.csv", usecols = catcols + numcols, dtype = dtype)
# d[f'd_{day} for day in range(1, 1914)]
# for d in range(1942, 1970):
#     col = 'd_' +str(d)
#     sales[col] = 0
# sales.head()

In [ ]:
#create dt
numcols = [f"d_{day}" for day in range(1000,1942)]
catcols = ['id', 'item_id', 'dept_id','store_id', 'cat_id', 'state_id']
dtype = {numcol:"int16" for numcol in numcols} 
dtype.update({col: "category" for col in catcols if col != "id"})


sales = pd.read_csv("../input/m5-forecasting-accuracy/sales_train_evaluation.csv", usecols = catcols + numcols, dtype = dtype)

for d in range(1942, 1970):
    col = 'd_' +str(d)
    sales[col] = 0
sales.head()
  
for col in catcols:
    if col != "id":
        sales[col] = sales[col].cat.codes.astype("int16")
        sales[col] -= sales[col].min()
sales.head()

In [ ]:
# m = []
# for x in range(495,1970):
#     x = str(x)
#     m.append(x)

In [ ]:
# #m = ['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id']
# sales.columns = ['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id']+ m

In [ ]:
def melt_and_merge(dt):
    prices = pd.read_csv("../input/m5-forecasting-accuracy/sell_prices.csv", dtype = PRICE_DTYPES)
    for col, col_dtype in PRICE_DTYPES.items():
        if col_dtype == "category":
            prices[col] = prices[col].cat.codes.astype("int16")
            prices[col] -= prices[col].min()
            
    cal = pd.read_csv("../input/m5-forecasting-accuracy/calendar.csv", dtype = CAL_DTYPES)
    cal["date"] = pd.to_datetime(cal["date"])
    for col, col_dtype in CAL_DTYPES.items():
        if col_dtype == "category":
            cal[col] = cal[col].cat.codes.astype("int16")
            cal[col] -= cal[col].min()
            
            
    catcols = ['id', 'item_id', 'dept_id','store_id', 'cat_id', 'state_id']
    dt = pd.melt(dt,
                id_vars = catcols,
                value_vars = [col for col in dt.columns if col.startswith("d_")],
                var_name = "d",
                value_name = "sales")
    
    dt = dt.merge(cal, on= "d", copy = False)
    dt = dt.merge(prices, on = ["store_id", "item_id", "wm_yr_wk"], copy = False)
    return dt

In [ ]:
def create_fea(dt):
    lags = [7, 28]
    lag_cols = [f"lag_{lag}" for lag in lags ]
    for lag, lag_col in zip(lags, lag_cols):
        dt[lag_col] = dt[["id","sales"]].groupby("id")["sales"].shift(lag)

    wins = [7, 28]
    for win in wins :
        for lag,lag_col in zip(lags, lag_cols):
            dt[f"rmean_{lag}_{win}"] = dt[["id", lag_col]].groupby("id")[lag_col].transform(lambda x : x.rolling(win).mean())
            #df['rolling_sold_mean'] = df.groupby(['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'])['sold'].transform(lambda x: x.rolling(window=7).mean()).astype(np.float16)
    
    dt['expanding_sold_mean'] = dt.groupby(['id'])['sales'].transform(lambda x: x.expanding(2).mean()).astype(np.float16)#accumulative 每次加下面一个
    
    dt['state_cat_sold_avg'] = dt.groupby(['state_id','cat_id'])['sales'].transform('mean').astype(np.float16)
    dt['state_dept_sold_avg'] = dt.groupby(['state_id','dept_id'])['sales'].transform('mean').astype(np.float16)
    
    dt['store_cat_sold_avg'] = dt.groupby(['store_id','cat_id'])['sales'].transform('mean').astype(np.float16)
    dt['store_dept_sold_avg'] = dt.groupby(['store_id','dept_id'])['sales'].transform('mean').astype(np.float16)
    
    
    dt['daily_avg_sales'] = dt.groupby(['id','d'])['sales'].transform('mean').astype(np.float16)
    dt['avg_sales'] = dt.groupby(['id'])['sales'].transform('mean').astype(np.float16)
    dt['selling_trend'] = (dt['daily_avg_sales'] - dt['avg_sales']).astype(np.float16)
    dt.drop(['daily_avg_sales','avg_sales'],axis=1,inplace=True)
    
    
    
    
    
#     date_features = {
        
#         "wday": "weekday",
#         "week": "weekofyear",
#         "month": "month",
#         "quarter": "quarter",
#         "year": "year",
#         "mday": "day",
# #         "ime": "is_month_end",
# #         "ims": "is_month_start",
#     }
    
    dt.drop(["wm_yr_wk", "weekday"], axis=1, inplace = True)
    
#     for date_feat_name, date_feat_func in date_features.items():
#         if date_feat_name in dt.columns:
#             dt[date_feat_name] = dt[date_feat_name].astype("int16")
#         else:
#             dt[date_feat_name] = getattr(dt["date"].dt, date_feat_func).astype("int16")
    return dt

In [ ]:
dt = melt_and_merge(sales)
dt.head()

In [ ]:
del sales; gc.collect()

In [ ]:
dt = create_fea(dt)
dt.head()

In [ ]:
dt.d = dt.d.apply(lambda x: x.split('_')[-1]).astype('int16')
dt.head()

In [ ]:
#dt.isnull().sum()

In [ ]:
X_train, y_train = dt[dt['d']<1914].drop('sales',axis=1), dt[dt['d']<1914]['sales']
X_valid, y_valid = dt[(dt['d']>=1914) & (dt['d']<1942)].drop('sales',axis=1), dt[(dt['d']>=1914) & (dt['d']<1942)]['sales']
X_test = dt[dt['d']>=1942].drop('sales',axis=1)
X_valid

In [ ]:
dt.drop(['date','id'], axis=1,inplace=True)
dt.head()

In [ ]:
params = {"objective" : "poisson",
          "metric" :"rmse",
          #"force_row_wise" : True, #enabling this is recommended when:
# the number of columns is large, or the total number of bins is large
# num_threads is large, e.g. >20
# you want to reduce memory cost
          "learning_rate" : 0.075, #learning_rate > 0.0Typical: 0.05.
#"sub_feature" : 0.8,#if you set it to 0.8, LightGBM will select 80% of features; alias 'feature_fraction' default as 1
          "sub_row" : 0.75, #0.0 < bagging_fraction <= 1.0
# like feature_fraction, but this will randomly select part of data without resampling
# can be used to speed up training
# can be used to deal with over-fitting
# Note: to enable bagging, bagging_freq should be set to a non zero value as well
#"bagging_fraction": 1 #to enable bagging, bagging_fraction should be set to value smaller than 1.0 as well
          "bagging_freq" : 1, #0 means disable bagging; k means perform bagging at every k iteration
    #to enable bagging, bagging_fraction should be set to value smaller than 1.0 as well. defaul as 0
          "lambda_l2" : 0.1, #alias reg_alpha, >=1, regularization
          "nthread" : 4, #real CPU cores
            #"metric": ["rmse"],
            #'verbosity': 1,
          'num_iterations' : 1200, #use smaller learning_rate with larger num_iterations. Also, you should use early_stopping_rounds if you go for higher num_iterations to stop your training when it is not learning anything useful.
          'early_stopping_rounds': 80, #The rule of thumb is to have it at 10% of your num_iterations.
    'num_leaves': 128, #1 < num_leaves <= 131072 max # of leaves in one tree, controling depth, overfitting.A higher value results in deeper trees.
    #num_leaves = 2^(max_depth); It is necessary to tune num_leaves with the max_depth together.
    "min_data_in_leaf": 100, #default as 20, >0, control overfitting. min_data_in_leaf depends on the number of training samples and num_leaves
}

In [ ]:
cat_feats = ['item_id', 'dept_id','store_id', 'cat_id', 'state_id'] + ["event_name_1", "event_name_2", "event_type_1", "event_type_2"]
train_data = lgb.Dataset(X_train , label = y_train, 
                         categorical_feature=cat_feats, free_raw_data=False)#which one is categorical feature
valid_data = lgb.Dataset(X_valid, label = y_valid,
                              categorical_feature=cat_feats,
                 free_raw_data=False)

In [ ]:
%%time

m_lgb = lgb.train(params, train_data, valid_sets = valid_data, verbose_eval=200) 

In [ ]:
m_lgb.save_model("model.lgb")

In [ ]:
y_test = m_lgb.predict(X_test)

In [ ]:
y_test = [round(a, 4) for a in y_test]
y_test

In [ ]:
result = np.reshape(y_test, (30490,28))
m = pd.DataFrame(result)
m

In [ ]:
m.columns=['id'] + ['F' + str(i + 1) for i in range(28)]
m
#f'F{i}' for i in range(1,29)

In [7]:
sub = pd.read_csv('../input/m5-forecasting-accuracy/sample_submission.csv')

In [ ]:
m['id']=sub.id[30491:]

In [3]:
sales = pd.read_csv('../input/m5-forecasting-accuracy/sales_train_evaluation.csv')

In [8]:
validation = sales[['id']+['d_' + str(i) for i in range(1914,1942)]]

validation['id']=sub.id[:30491]
validation.columns=['id'] + ['F' + str(i + 1) for i in range(28)]
validation

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,id,F1,F2,F3,F4,F5,F6,F7,F8,F9,...,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
0,HOBBIES_1_001_CA_1_validation,0,0,0,2,0,3,5,0,0,...,2,4,0,0,0,0,3,3,0,1
1,HOBBIES_1_002_CA_1_validation,0,1,0,0,0,0,0,0,0,...,0,1,2,1,1,0,0,0,0,0
2,HOBBIES_1_003_CA_1_validation,0,0,1,1,0,2,1,0,0,...,1,0,2,0,0,0,2,3,0,1
3,HOBBIES_1_004_CA_1_validation,0,0,1,2,4,1,6,4,0,...,1,1,0,4,0,1,3,0,2,6
4,HOBBIES_1_005_CA_1_validation,1,0,2,3,1,0,3,2,3,...,0,0,0,2,1,0,0,2,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30485,FOODS_3_823_WI_3_validation,0,0,0,2,2,0,0,0,2,...,1,0,3,0,1,1,0,0,1,1
30486,FOODS_3_824_WI_3_validation,0,1,1,1,0,0,0,0,1,...,0,0,0,0,0,0,1,0,1,0
30487,FOODS_3_825_WI_3_validation,0,0,1,1,0,2,1,1,0,...,0,0,1,2,0,1,0,1,0,2
30488,FOODS_3_826_WI_3_validation,1,3,0,1,2,1,0,2,1,...,1,1,1,4,6,0,1,1,1,0


In [ ]:
submission = pd.concat([validation, m], axis=0, sort=False)
submission.head()

In [ ]:
submission.to_csv('submission.csv', index=False)